## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=2)

# Get the date one week ago
one_week_ago = today - timedelta(days=8)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

May. 23rd May. 31st


In [3]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,June-2-2025,Condo/Co-op,1300 Brickell Bay Dr #2404,Miami,FL,33131.0,640000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1300-Brickell-...,MARMLS,A11763087,N,Y,25.760372,-80.190197


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df2 = df[(df['SOLD DATE'] > one_week_ago) & (df['SOLD DATE'] < today)]

In [10]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-05-30    50
2025-05-28    29
2025-05-27    25
2025-05-29    24
2025-05-26     1
Name: count, dtype: int64

In [11]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [12]:
df_top_ten = df2.head(10)

In [14]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
20,PAST SALE,2025-05-28,Condo/Co-op,9001 Collins Ave Unit S-903,Surfside,FL,33154.0,19500000.0,4.0,5.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9001-Collin...,MARMLS,A11778858,N,Y,25.876848,-80.121466
104,PAST SALE,2025-05-30,Condo/Co-op,17901 Collins Ave Unit TS4205,Sunny Isles Beach,FL,33160.0,17875000.0,6.0,8.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11676098,N,Y,25.941865,-80.120490
113,PAST SALE,2025-05-28,Condo/Co-op,7872 Fisher Island Dr #7872,Miami Beach,FL,33109.0,12300000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7872-Fis...,MARMLS,A11615008,N,Y,25.759774,-80.137270
34,PAST SALE,2025-05-28,Condo/Co-op,500 Alton Rd #2501,Miami Beach,FL,33139.0,9000000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11761445,N,Y,25.775127,-80.141257
25,PAST SALE,2025-05-27,Condo/Co-op,9001 Collins Ave Unit S-606,Surfside,FL,33154.0,6850000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9001-Collin...,MARMLS,A11759743,N,Y,25.876848,-80.121466
126,PAST SALE,2025-05-27,Condo/Co-op,1000 Biscayne Blvd #3001,Miami,FL,33132.0,5600000.0,4.0,5.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Biscayne-...,MARMLS,A11586563,N,Y,25.784171,-80.190078
92,PAST SALE,2025-05-27,Condo/Co-op,100 S Pointe Dr #2107,Miami Beach,FL,33139.0,5250000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11729072,N,Y,25.767180,-80.132822
66,PAST SALE,2025-05-29,Condo/Co-op,101 20th St #2208,Miami Beach,FL,33139.0,4275000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/101-20th...,MARMLS,A11734833,N,Y,25.795451,-80.128287
118,PAST SALE,2025-05-28,Condo/Co-op,1470 16th St Ph -1,Miami Beach,FL,33139.0,4000000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1470-16t...,MARMLS,A11495252,N,Y,25.788734,-80.144759
36,PAST SALE,2025-05-28,Condo/Co-op,500 Alton Rd #1401,Miami Beach,FL,33139.0,3450000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11720287,N,Y,25.775127,-80.141257


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [15]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [16]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [17]:
response_df = pd.DataFrame(response_list)

In [18]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [19]:
df_top_ten = merged_df

In [20]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-05-28,Condo/Co-op,9001 Collins Ave Unit S-903,Surfside,FL,33154.0,19500000.0,4.0,5.5,...,-80.121466,https://www.redfin.com/FL/Surfside/9001-Collin...,Ximena Penuela,"Fort Realty, LLC",None,None,Ximena Penuela,"Fort Realty, LLC",None,None
1,PAST SALE,2025-05-30,Condo/Co-op,17901 Collins Ave Unit TS4205,Sunny Isles Beach,FL,33160.0,17875000.0,6.0,8.5,...,-80.120490,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Ryan Mendell,Maxwell E Realty Inc,None,None,Yansy Checa,Douglas Elliman,None,None
2,PAST SALE,2025-05-28,Condo/Co-op,7872 Fisher Island Dr #7872,Miami Beach,FL,33109.0,12300000.0,3.0,3.5,...,-80.137270,https://www.redfin.com/FL/Miami-Beach/7872-Fis...,Denise Levine,"Denise LeVine Realty, Inc.",None,None,Maria Oneide Gioia,Luxury Living Realty,None,None
3,PAST SALE,2025-05-28,Condo/Co-op,500 Alton Rd #2501,Miami Beach,FL,33139.0,9000000.0,3.0,3.5,...,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Angelica Garcia,Douglas Elliman,None,None,Stefania Riverin,One Sotheby's Int'l Realty,None,None
4,PAST SALE,2025-05-27,Condo/Co-op,9001 Collins Ave Unit S-606,Surfside,FL,33154.0,6850000.0,2.0,2.5,...,-80.121466,https://www.redfin.com/FL/Surfside/9001-Collin...,Anthony Nucci,Douglas Elliman,None,None,Marco Tine,Serhant,None,None
5,PAST SALE,2025-05-27,Condo/Co-op,1000 Biscayne Blvd #3001,Miami,FL,33132.0,5600000.0,4.0,5.5,...,-80.190078,https://www.redfin.com/FL/Miami/1000-Biscayne-...,Elena Sanilevich,Cervera Real Estate Inc.,Nickel Goeseke,Cervera Real Estate Inc.,Jonathan Mann,Coldwell Banker Realty,None,None
6,PAST SALE,2025-05-27,Condo/Co-op,100 S Pointe Dr #2107,Miami Beach,FL,33139.0,5250000.0,2.0,2.5,...,-80.132822,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,Constantin Gorges,"Compass Florida, LLC.",None,None,Salesperson Non Member,Non-MLS Member,None,None
7,PAST SALE,2025-05-29,Condo/Co-op,101 20th St #2208,Miami Beach,FL,33139.0,4275000.0,2.0,2.0,...,-80.128287,https://www.redfin.com/FL/Miami-Beach/101-20th...,Michael Duchon,The Corcoran Group,Eloy Carmenate,The Corcoran Group,Benjamin Moss,"Compass Florida, LLC.",None,None
8,PAST SALE,2025-05-28,Condo/Co-op,1470 16th St Ph -1,Miami Beach,FL,33139.0,4000000.0,3.0,2.5,...,-80.144759,https://www.redfin.com/FL/Miami-Beach/1470-16t...,Eloy Carmenate,The Corcoran Group,Sofia Joelsson,The Corcoran Group,Zachary Akers,One Sotheby's International Realty,None,None
9,PAST SALE,2025-05-28,Condo/Co-op,500 Alton Rd #1401,Miami Beach,FL,33139.0,3450000.0,3.0,3.5,...,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Angelica Garcia,Douglas Elliman,None,None,Anca Mirescu,Douglas Elliman,None,None


## Current Week's Values

In [21]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

129


In [22]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$1,173,360


In [23]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$657


In [24]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$151,363,476


In [25]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [26]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PSF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input county name:


 Miami-Dade


Input Previous Week Condo Sales Total (number units sold):
86
-----------
Input Previous Week Condo Average Sales Price:
1023844
-----------
Input Previous Week Condo Average PSF:
542
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
88050620.0


In [27]:
last_week

,Condo Sales,Average Sale Price,Average PSF,Last Week Total Dollar Volume
0,86,1023844,542,88050620.0


In [28]:
last_week['Last Week Total Dollar Volume'].iloc[0]

88050620.0

In [29]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0          129             1173360          657                    151363476.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [30]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [31]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [32]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [33]:
pd.set_option('display.max_columns',None)

In [34]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-05-28,Condo/Co-op,9001 Collins Ave Unit S-903,Surfside,FL,33154.0,19500000.0,4.0,5.5,SURF CLUB CONDO,3844.0,NaN,2017.0,NaN,5073.0,18320.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9001-Collin...,MARMLS,A11778858,N,Y,25.876848,-80.121466,https://www.redfin.com/FL/Surfside/9001-Collin...,Ximena Penuela,"Fort Realty, LLC",None,None,Ximena Penuela,"Fort Realty, LLC",None,None,1,orange


In [35]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [36]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-05-28,Condo/Co-op,9001 Collins Ave Unit S-903,Surfside,FL,33154.0,19500000.0,4.0,5.5,SURF CLUB CONDO,3844.0,NaN,2017.0,NaN,5073.0,18320.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9001-Collin...,MARMLS,A11778858,N,Y,25.876848,-80.121466,https://www.redfin.com/FL/Surfside/9001-Collin...,Ximena Penuela,"Fort Realty, LLC",None,None,Ximena Penuela,"Fort Realty, LLC",None,None,1,orange
1,PAST SALE,2025-05-30,Condo/Co-op,17901 Collins Ave Unit TS4205,Sunny Isles Beach,FL,33160.0,17875000.0,6.0,8.5,Estates at Acqualina,8139.0,NaN,2022.0,NaN,2196.0,18252.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11676098,N,Y,25.941865,-80.120490,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Ryan Mendell,Maxwell E Realty Inc,None,None,Yansy Checa,Douglas Elliman,None,None,2,blue
2,PAST SALE,2025-05-28,Condo/Co-op,7872 Fisher Island Dr #7872,Miami Beach,FL,33109.0,12300000.0,3.0,3.5,OCEANSIDE FISHER ISL COND,4590.0,NaN,1990.0,NaN,2680.0,11716.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7872-Fis...,MARMLS,A11615008,N,Y,25.759774,-80.137270,https://www.redfin.com/FL/Miami-Beach/7872-Fis...,Denise Levine,"Denise LeVine Realty, Inc.",None,None,Maria Oneide Gioia,Luxury Living Realty,None,None,3,blue
3,PAST SALE,2025-05-28,Condo/Co-op,500 Alton Rd #2501,Miami Beach,FL,33139.0,9000000.0,3.0,3.5,Miami Beach,3972.0,NaN,2024.0,NaN,2266.0,4422.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11761445,N,Y,25.775127,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Angelica Garcia,Douglas Elliman,None,None,Stefania Riverin,One Sotheby's Int'l Realty,None,None,4,blue
4,PAST SALE,2025-05-27,Condo/Co-op,9001 Collins Ave Unit S-606,Surfside,FL,33154.0,6850000.0,2.0,2.5,SURF CLUB CONDO,1753.0,NaN,2017.0,NaN,3908.0,9609.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9001-Collin...,MARMLS,A11759743,N,Y,25.876848,-80.121466,https://www.redfin.com/FL/Surfside/9001-Collin...,Anthony Nucci,Douglas Elliman,None,None,Marco Tine,Serhant,None,None,5,blue
5,PAST SALE,2025-05-27,Condo/Co-op,1000 Biscayne Blvd #3001,Miami,FL,33132.0,5600000.0,4.0,5.5,ONE THOUSAND MUSEUM CONDO,4600.0,NaN,2019.0,NaN,1217.0,9838.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Biscayne-...,MARMLS,A11586563,N,Y,25.784171,-80.190078,https://www.redfin.com/FL/Miami/1000-Biscayne-...,Elena Sanilevich,Cervera Real Estate Inc.,Nickel Goeseke,Cervera Real Estate Inc.,Jonathan Mann,Coldwell Banker Realty,None,None,6,blue
6,PAST SALE,2025-05-27,Condo/Co-op,100 S Pointe Dr #2107,Miami Beach,FL,33139.0,5250000.0,2.0,2.5,CONTINUUM ON SOUTH BEACH,1870.0,NaN,2002.0,NaN,2807.0,4580.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11729072,N,Y,25.767180,-80.132822,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,Constantin Gorges,"Compass Florida, LLC.",None,None,Salesperson Non Member,Non-MLS Member,None,None,7,blue
7,PAST SALE,2025-05-29,Condo/Co-op,101 20th St #2208,Miami Beach,FL,33139.0,4275000.0,2.0,2.0,SETAI RESORT & RESIDENCES,1242.0,NaN,2005.0,NaN,3442.0,11294.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/101-20th...,MARMLS,A11734833,N,Y,25.795451,-80.128287,https://www.redfin.com/FL/Miami-Beach/101-20th...,Michael Duchon,The Corcoran Group,Eloy Carmenate,The Corcoran Group,Benjamin Moss,"Compass Florida, LLC.",None,None,8,blue
8,PAST SALE,2025-05-28,Condo/Co-op,1470 16th St Ph -1,Miami Beach,FL,33139.0,4000000.0,3.0,2.5,CAPRI SOUTH BEACH CONDO,1974.0,NaN,2008.0,NaN,2026.0,4534.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-B

In [37]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [38]:
m.save('index.html')

## Data snagger

In [39]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [40]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-05-28,Condo/Co-op,9001 Collins Ave Unit S-903,Surfside,FL,33154.0,19500000.0,4.0,5.5,SURF CLUB CONDO,3844.0,NaN,2017.0,NaN,5073.0,18320.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9001-Collin...,MARMLS,A11778858,N,Y,25.876848,-80.121466,https://www.redfin.com/FL/Surfside/9001-Collin...,Ximena Penuela,"Fort Realty, LLC",None,None,Ximena Penuela,"Fort Realty, LLC",None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [41]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDade_County_condo_sales_week_ending_06022025 


# CREATE TEMPLATE 

In [42]:
muni_set = set(df_top_ten['CITY'])

In [43]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [44]:
df_top_ten.reset_index(inplace=True,drop=True)

In [45]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [46]:
top_sale

'SURF CLUB CONDO at 9001 Collins Ave Unit S-903 in Surfside'

In [47]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [48]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [49]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-05-28,Condo/Co-op,9001 Collins Ave Unit S-903,Surfside,FL,33154.0,19500000.0,4.0,5.5,SURF CLUB CONDO,3844.0,NaN,2017.0,NaN,5073.0,18320.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9001-Collin...,MARMLS,A11778858,N,Y,25.876848,-80.121466,https://www.redfin.com/FL/Surfside/9001-Collin...,Ximena Penuela,"Fort Realty, LLC",None,None,Ximena Penuela,"Fort Realty, LLC",None,None,1,orange,SURF CLUB CONDO at 9001 Collins Ave Unit S-903...
1,PAST SALE,2025-05-30,Condo/Co-op,17901 Collins Ave Unit TS4205,Sunny Isles Beach,FL,33160.0,17875000.0,6.0,8.5,Estates at Acqualina,8139.0,NaN,2022.0,NaN,2196.0,18252.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11676098,N,Y,25.941865,-80.120490,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Ryan Mendell,Maxwell E Realty Inc,None,None,Yansy Checa,Douglas Elliman,None,None,2,blue,Estates at Acqualina at 17901 Collins Ave Unit...
2,PAST SALE,2025-05-28,Condo/Co-op,7872 Fisher Island Dr #7872,Miami Beach,FL,33109.0,12300000.0,3.0,3.5,OCEANSIDE FISHER ISL COND,4590.0,NaN,1990.0,NaN,2680.0,11716.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7872-Fis...,MARMLS,A11615008,N,Y,25.759774,-80.137270,https://www.redfin.com/FL/Miami-Beach/7872-Fis...,Denise Levine,"Denise LeVine Realty, Inc.",None,None,Maria Oneide Gioia,Luxury Living Realty,None,None,3,blue,OCEANSIDE FISHER ISL COND at 7872 Fisher Islan...
3,PAST SALE,2025-05-28,Condo/Co-op,500 Alton Rd #2501,Miami Beach,FL,33139.0,9000000.0,3.0,3.5,Miami Beach,3972.0,NaN,2024.0,NaN,2266.0,4422.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11761445,N,Y,25.775127,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Angelica Garcia,Douglas Elliman,None,None,Stefania Riverin,One Sotheby's Int'l Realty,None,None,4,blue,Miami Beach at 500 Alton Rd #2501 in Miami Beach
4,PAST SALE,2025-05-27,Condo/Co-op,9001 Collins Ave Unit S-606,Surfside,FL,33154.0,6850000.0,2.0,2.5,SURF CLUB CONDO,1753.0,NaN,2017.0,NaN,3908.0,9609.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9001-Collin...,MARMLS,A11759743,N,Y,25.876848,-80.121466,https://www.redfin.com/FL/Surfside/9001-Collin...,Anthony Nucci,Douglas Elliman,None,None,Marco Tine,Serhant,None,None,5,blue,SURF CLUB CONDO at 9001 Collins Ave Unit S-606...
5,PAST SALE,2025-05-27,Condo/Co-op,1000 Biscayne Blvd #3001,Miami,FL,33132.0,5600000.0,4.0,5.5,ONE THOUSAND MUSEUM CONDO,4600.0,NaN,2019.0,NaN,1217.0,9838.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Biscayne-...,MARMLS,A11586563,N,Y,25.784171,-80.190078,https://www.redfin.com/FL/Miami/1000-Biscayne-...,Elena Sanilevich,Cervera Real Estate Inc.,Nickel Goeseke,Cervera Real Estate Inc.,Jonathan Mann,Coldwell Banker Realty,None,None,6,blue,ONE THOUSAND MUSEUM CONDO at 1000 Biscayne Blv...
6,PAST SALE,2025-05-27,Condo/Co-op,100 S Pointe Dr #2107,Miami Beach,FL,33139.0,5250000.0,2.0,2.5,CONTINUUM ON SOUTH BEACH,1870.0,NaN,2002.0,NaN,2807.0,4580.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11729072,N,Y,25.767180,-80.132822,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,Constantin Gorges,"Compass Florida, LLC.",None,None,Salesperson Non Member,Non-MLS Member,None,None,7,blue,CONTINUUM ON SOUTH BEACH at 100 S Pointe Dr #2...
7,PAST SALE,2025-05-29,Condo/Co-op,101 20th St #2208,Miami Beach,FL,33139.0,4275000.0,2.0,2.0,SETAI RESORT & RESIDENCES,1242.0,NaN,2005.0,NaN,3442.0,11294.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/101-20th...,MARMLS,A11734833,N,Y,25.795451,-80.128287,https://www.red

In [50]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: SURF CLUB CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $3,450,000 to $19,500,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami Beach, Sunny Isles Beach, Surfside, Miami, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 129 condo sales totaling $151,363,476 million from May. 23rd to May. 31st. The previous week, brokers closed 86 condo sales totaling $88,050,620.

Last week’s units sold for an average of $1,173,360, higher than the $1,023,844 average price from the pr

In [51]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [52]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [53]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [54]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-05-28,Condo/Co-op,9001 Collins Ave Unit S-903,Surfside,FL,33154.0,19500000.0,4.0,5.5,SURF CLUB CONDO,3844.0,NaN,2017.0,NaN,5073.0,18320.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9001-Collin...,MARMLS,A11778858,N,Y,25.876848,-80.121466,https://www.redfin.com/FL/Surfside/9001-Collin...,Ximena Penuela,"Fort Realty, LLC",None,None,Ximena Penuela,"Fort Realty, LLC",None,None,1,orange,SURF CLUB CONDO at 9001 Collins Ave Unit S-903...
1,PAST SALE,2025-05-30,Condo/Co-op,17901 Collins Ave Unit TS4205,Sunny Isles Beach,FL,33160.0,17875000.0,6.0,8.5,Estates at Acqualina,8139.0,NaN,2022.0,NaN,2196.0,18252.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11676098,N,Y,25.941865,-80.120490,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Ryan Mendell,Maxwell E Realty Inc,None,None,Yansy Checa,Douglas Elliman,None,None,2,blue,Estates at Acqualina at 17901 Collins Ave Unit...
2,PAST SALE,2025-05-28,Condo/Co-op,7872 Fisher Island Dr #7872,Miami Beach,FL,33109.0,12300000.0,3.0,3.5,OCEANSIDE FISHER ISL COND,4590.0,NaN,1990.0,NaN,2680.0,11716.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7872-Fis...,MARMLS,A11615008,N,Y,25.759774,-80.137270,https://www.redfin.com/FL/Miami-Beach/7872-Fis...,Denise Levine,"Denise LeVine Realty, Inc.",None,None,Maria Oneide Gioia,Luxury Living Realty,None,None,3,blue,OCEANSIDE FISHER ISL COND at 7872 Fisher Islan...
3,PAST SALE,2025-05-28,Condo/Co-op,500 Alton Rd #2501,Miami Beach,FL,33139.0,9000000.0,3.0,3.5,Miami Beach,3972.0,NaN,2024.0,NaN,2266.0,4422.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11761445,N,Y,25.775127,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Angelica Garcia,Douglas Elliman,None,None,Stefania Riverin,One Sotheby's Int'l Realty,None,None,4,blue,Miami Beach at 500 Alton Rd #2501 in Miami Beach
4,PAST SALE,2025-05-27,Condo/Co-op,9001 Collins Ave Unit S-606,Surfside,FL,33154.0,6850000.0,2.0,2.5,SURF CLUB CONDO,1753.0,NaN,2017.0,NaN,3908.0,9609.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9001-Collin...,MARMLS,A11759743,N,Y,25.876848,-80.121466,https://www.redfin.com/FL/Surfside/9001-Collin...,Anthony Nucci,Douglas Elliman,None,None,Marco Tine,Serhant,None,None,5,blue,SURF CLUB CONDO at 9001 Collins Ave Unit S-606...
5,PAST SALE,2025-05-27,Condo/Co-op,1000 Biscayne Blvd #3001,Miami,FL,33132.0,5600000.0,4.0,5.5,ONE THOUSAND MUSEUM CONDO,4600.0,NaN,2019.0,NaN,1217.0,9838.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Biscayne-...,MARMLS,A11586563,N,Y,25.784171,-80.190078,https://www.redfin.com/FL/Miami/1000-Biscayne-...,Elena Sanilevich,Cervera Real Estate Inc.,Nickel Goeseke,Cervera Real Estate Inc.,Jonathan Mann,Coldwell Banker Realty,None,None,6,blue,ONE THOUSAND MUSEUM CONDO at 1000 Biscayne Blv...
6,PAST SALE,2025-05-27,Condo/Co-op,100 S Pointe Dr #2107,Miami Beach,FL,33139.0,5250000.0,2.0,2.5,CONTINUUM ON SOUTH BEACH,1870.0,NaN,2002.0,NaN,2807.0,4580.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11729072,N,Y,25.767180,-80.132822,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,Constantin Gorges,"Compass Florida, LLC.",None,None,Salesperson Non Member,Non-MLS Member,None,None,7,blue,CONTINUUM ON SOUTH BEACH at 100 S Pointe Dr #2...
7,PAST SALE,2025-05-29,Condo/Co-op,101 20th St #2208,Miami Beach,FL,33139.0,4275000.0,2.0,2.0,SETAI RESORT & RESIDENCES,1242.0,NaN,2005.0,NaN,3442.0,11294.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/101-20th...,MARMLS,A11734833,N,Y,25.795451,-80.128287,https://www.red

In [55]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: SURF CLUB CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $3,450,000 to $19,500,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami Beach, Sunny Isles Beach, Surfside, Miami, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 129 condo sales totaling $151,363,476 million from May. 23rd to May. 31st. The previous week, brokers closed 86 condo sales totaling $88,050,620.

Last week’s units sold for an average of $1,173,360, higher than the $1,023,844 average price from the pr

In [63]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Surfside/9001-Collins-Ave-33154/unit-S903/home/79133349


In [64]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Sunny-Isles-Beach/17901-Collins-Ave-33160/unit-TS4205/home/183119605


In [58]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Miami-Beach/500-Alton-Rd-33139/unit-1401/home/194157817


In [59]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Miami-Beach/1470-16th-St-33139/unit-1/home/42679115


In [60]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Surfside/9001-Collins-Ave-33154/unit-S903/home/79133349


In [61]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Miami/1000-Biscayne-Blvd-33132/unit-3001/home/168724901


In [62]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-05-28,Condo/Co-op,9001 Collins Ave Unit S-903,Surfside,FL,33154.0,19500000.0,4.0,5.5,SURF CLUB CONDO,3844.0,NaN,2017.0,NaN,5073.0,18320.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9001-Collin...,MARMLS,A11778858,N,Y,25.876848,-80.121466,https://www.redfin.com/FL/Surfside/9001-Collin...,Ximena Penuela,"Fort Realty, LLC",None,None,Ximena Penuela,"Fort Realty, LLC",None,None,1,orange,SURF CLUB CONDO at 9001 Collins Ave Unit S-903...
1,PAST SALE,2025-05-30,Condo/Co-op,17901 Collins Ave Unit TS4205,Sunny Isles Beach,FL,33160.0,17875000.0,6.0,8.5,Estates at Acqualina,8139.0,NaN,2022.0,NaN,2196.0,18252.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11676098,N,Y,25.941865,-80.120490,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Ryan Mendell,Maxwell E Realty Inc,None,None,Yansy Checa,Douglas Elliman,None,None,2,blue,Estates at Acqualina at 17901 Collins Ave Unit...
2,PAST SALE,2025-05-28,Condo/Co-op,7872 Fisher Island Dr #7872,Miami Beach,FL,33109.0,12300000.0,3.0,3.5,OCEANSIDE FISHER ISL COND,4590.0,NaN,1990.0,NaN,2680.0,11716.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7872-Fis...,MARMLS,A11615008,N,Y,25.759774,-80.137270,https://www.redfin.com/FL/Miami-Beach/7872-Fis...,Denise Levine,"Denise LeVine Realty, Inc.",None,None,Maria Oneide Gioia,Luxury Living Realty,None,None,3,blue,OCEANSIDE FISHER ISL COND at 7872 Fisher Islan...
3,PAST SALE,2025-05-28,Condo/Co-op,500 Alton Rd #2501,Miami Beach,FL,33139.0,9000000.0,3.0,3.5,Miami Beach,3972.0,NaN,2024.0,NaN,2266.0,4422.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11761445,N,Y,25.775127,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Angelica Garcia,Douglas Elliman,None,None,Stefania Riverin,One Sotheby's Int'l Realty,None,None,4,blue,Miami Beach at 500 Alton Rd #2501 in Miami Beach
4,PAST SALE,2025-05-27,Condo/Co-op,9001 Collins Ave Unit S-606,Surfside,FL,33154.0,6850000.0,2.0,2.5,SURF CLUB CONDO,1753.0,NaN,2017.0,NaN,3908.0,9609.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9001-Collin...,MARMLS,A11759743,N,Y,25.876848,-80.121466,https://www.redfin.com/FL/Surfside/9001-Collin...,Anthony Nucci,Douglas Elliman,None,None,Marco Tine,Serhant,None,None,5,blue,SURF CLUB CONDO at 9001 Collins Ave Unit S-606...
5,PAST SALE,2025-05-27,Condo/Co-op,1000 Biscayne Blvd #3001,Miami,FL,33132.0,5600000.0,4.0,5.5,ONE THOUSAND MUSEUM CONDO,4600.0,NaN,2019.0,NaN,1217.0,9838.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Biscayne-...,MARMLS,A11586563,N,Y,25.784171,-80.190078,https://www.redfin.com/FL/Miami/1000-Biscayne-...,Elena Sanilevich,Cervera Real Estate Inc.,Nickel Goeseke,Cervera Real Estate Inc.,Jonathan Mann,Coldwell Banker Realty,None,None,6,blue,ONE THOUSAND MUSEUM CONDO at 1000 Biscayne Blv...
6,PAST SALE,2025-05-27,Condo/Co-op,100 S Pointe Dr #2107,Miami Beach,FL,33139.0,5250000.0,2.0,2.5,CONTINUUM ON SOUTH BEACH,1870.0,NaN,2002.0,NaN,2807.0,4580.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11729072,N,Y,25.767180,-80.132822,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,Constantin Gorges,"Compass Florida, LLC.",None,None,Salesperson Non Member,Non-MLS Member,None,None,7,blue,CONTINUUM ON SOUTH BEACH at 100 S Pointe Dr #2...
7,PAST SALE,2025-05-29,Condo/Co-op,101 20th St #2208,Miami Beach,FL,33139.0,4275000.0,2.0,2.0,SETAI RESORT & RESIDENCES,1242.0,NaN,2005.0,NaN,3442.0,11294.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/101-20th...,MARMLS,A11734833,N,Y,25.795451,-80.128287,https://www.red

## Time on Market Calculator

In [66]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2025, 5, 1) ## List (Earlier) date
date2 = datetime(2025, 5, 30) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

29


## Clean CSV for Datawrapper Chart

In [67]:
chart_df = df_top_ten

In [68]:
chart_df = chart_df.fillna(" ")

In [69]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [70]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [71]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [72]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [73]:
chart_df['$/SQUARE FEET'].astype(float)

0    5073.0
1    2196.0
2    2680.0
3    2266.0
4    3908.0
5    1217.0
6    2807.0
7    3442.0
8    2026.0
9    1584.0
Name: $/SQUARE FEET, dtype: float64

In [74]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [75]:
csv_date_string = today.strftime("%m_%d_%Y")

In [76]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [77]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-05-28,9001 Collins Ave Unit S-903 in Surfside,"$19,500,000",4,5.5,"3,844",2017,"$5,073",MARMLS,A11778858,"Ximena Penuela Fort Realty, LLC","Ximena Penuela Fort Realty, LLC"
1,2025-05-30,17901 Collins Ave Unit TS4205 in Sunny Isles B...,"$17,875,000",6,8.5,"8,139",2022,"$2,196",MARMLS,A11676098,Ryan Mendell Maxwell E Realty Inc,Yansy Checa Douglas Elliman
2,2025-05-28,7872 Fisher Island Dr #7872 in Miami Beach,"$12,300,000",3,3.5,"4,590",1990,"$2,680",MARMLS,A11615008,"Denise Levine Denise LeVine Realty, Inc.",Maria Oneide Gioia Luxury Living Realty
3,2025-05-28,500 Alton Rd #2501 in Miami Beach,"$9,000,000",3,3.5,"3,972",2024,"$2,266",MARMLS,A11761445,Angelica Garcia Douglas Elliman,Stefania Riverin One Sotheby's Int'l Realty
4,2025-05-27,9001 Collins Ave Unit S-606 in Surfside,"$6,850,000",2,2.5,"1,753",2017,"$3,908",MARMLS,A11759743,Anthony Nucci Douglas Elliman,Marco Tine Serhant
5,2025-05-27,1000 Biscayne Blvd #3001 in Miami,"$5,600,000",4,5.5,"4,600",2019,"$1,217",MARMLS,A11586563,Elena Sanilevich Cervera Real Estate Inc. Nick...,Jonathan Mann Coldwell Banker Realty
6,2025-05-27,100 S Pointe Dr #2107 in Miami Beach,"$5,250,000",2,2.5,"1,870",2002,"$2,807",MARMLS,A11729072,"Constantin Gorges Compass Florida, LLC.",Salesperson Non Member Non-MLS Member
7,2025-05-29,101 20th St #2208 in Miami Beach,"$4,275,000",2,2.0,"1,242",2005,"$3,442",MARMLS,A11734833,Michael Duchon The Corcoran Group Eloy Carmena...,"Benjamin Moss Compass Florida, LLC."
8,2025-05-28,1470 16th St Ph -1 in Miami Beach,"$4,000,000",3,2.5,"1,974",2008,"$2,026",MARMLS,A11495252,Eloy Carmenate The Corcoran Group Sofia Joelss...,Zachary Akers One Sotheby's International Real...
9,2025-05-28,500 Alton Rd #1401 in Miami Beach,"$3,450,000",3,3.5,"2,178",2024,"$1,584",MARMLS,A11720287,Angelica Garcia Douglas Elliman,Anca Mirescu Douglas Elliman
